## Lenet 5 con MNIST

Imports

In [1]:
! pip install -q tensorflow-model-optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 67.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.1 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.25.1 which is incompatible.


In [2]:
import tempfile
import os
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, losses
import tensorflow_model_optimization as tfmot

Load dataset

In [3]:
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images.shape

11490434/11490434 [==============================] - 1s 0us/step


(60000, 28, 28)

Normalize the input image so that each pixel value is between 0 to 1.

In [4]:
train_images = tf.pad(train_images, [[0, 0], [2,2], [2,2]])/255
test_images = tf.pad(test_images, [[0, 0], [2,2], [2,2]])/255
train_images.shape

TensorShape([60000, 32, 32])

Most CNN’s accept 4-dimensional tensors as inputs having the dimensions of batch size, height, width, and channel. Since MNIST images are grayscale, the last dimension does not necessarily exist. We need to expand the tensor and create a dummy dimension at axis number 3. (Recall that the tensor initially had axis 0, 1, and 2.)

In [5]:
train_images = tf.expand_dims(train_images, axis=3, name=None)
test_images = tf.expand_dims(test_images, axis=3, name=None)
train_images.shape

TensorShape([60000, 32, 32, 1])

The last 2000 samples of the training set are reserved for the validation set.

In [6]:
val_images = train_images[-2000:,:,:,:]
val_labels = train_labels[-2000:]
train_images = train_images[:-2000,:,:,:]
train_labels = train_labels[:-2000]

Define the model architecture.

In [7]:
model = keras.models.load_model('model.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d (AverageP  (None, 14, 14, 6)        0         
 ooling2D)                                                       
                                                                 
 activation (Activation)     (None, 14, 14, 6)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_1 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                                     
                                                                 
 activation_1 (Activation)   (None, 5, 5, 16)          0

Pruning

In [8]:
quantize_model = tfmot.quantization.keras.quantize_model

In [9]:
# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLay  (None, 32, 32, 1)        3         
 er)                                                             
                                                                 
 quant_conv2d (QuantizeWrapp  (None, 28, 28, 6)        171       
 erV2)                                                           
                                                                 
 quant_average_pooling2d (Qu  (None, 14, 14, 6)        3         
 antizeWrapperV2)                                                
                                                                 
 quant_activation (QuantizeW  (None, 14, 14, 6)        1         
 rapperV2)                                                       
                                                                 
 quant_conv2d_1 (QuantizeWra  (None, 10, 10, 16)       2

Train model

In [10]:
q_aware_model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])
history = q_aware_model.fit(train_images, train_labels, batch_size=64, epochs=40, validation_data=(val_images, val_labels))

Epoch 1/40
907/907 [==============================] - 55s 56ms/step - loss: 0.0515 - accuracy: 0.9845 - val_loss: 0.0613 - val_accuracy: 0.9885
Epoch 2/40
907/907 [==============================] - 46s 50ms/step - loss: 0.0426 - accuracy: 0.9860 - val_loss: 0.0670 - val_accuracy: 0.9825
Epoch 3/40
907/907 [==============================] - 47s 51ms/step - loss: 0.0406 - accuracy: 0.9866 - val_loss: 0.0612 - val_accuracy: 0.9900
Epoch 4/40
907/907 [==============================] - 47s 52ms/step - loss: 0.0393 - accuracy: 0.9865 - val_loss: 0.0658 - val_accuracy: 0.9885
Epoch 5/40
907/907 [==============================] - 46s 51ms/step - loss: 0.0406 - accuracy: 0.9862 - val_loss: 0.0730 - val_accuracy: 0.9860
Epoch 6/40
907/907 [==============================] - 46s 50ms/step - loss: 0.0367 - accuracy: 0.9881 - val_loss: 0.0788 - val_accuracy: 0.9840
Epoch 7/40
907/907 [==============================] - 46s 51ms/step - loss: 0.0364 - accuracy: 0.9882 - val_loss: 0.0739 - val_accuracy:

Evaluate

In [13]:
start = time.time()
accuracy = q_aware_model.evaluate(test_images, test_labels)
end = time.time()
print("Accuracy :",accuracy[1])
print("Images per second :", int(test_images.shape[0]/(end - start)))

313/313 [==============================] - 3s 11ms/step - loss: 0.0561 - accuracy: 0.9833
Accuracy : 0.983299970626831
Images per second : 1923


In [15]:
# Guardar el Modelo
q_aware_model.save('model_quat.h5')
